In [1]:
#30/7/24 Creates a K fold RF model for each file in a folder provided they are Fingerprints

#3/7/24 For the implementation of K-Fold RF using combined Chemopy Mutagen Data

import os
import pandas as pd
import numpy as np
import math
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sys
sys.path.insert(1, '/Users/james/Documents/Honours/Python')
from Function_Repo import metriccalc 
from Function_Repo import setbalance
import math

seed = 81
#Data import
datasets = []
index = []
directory = '/Users/james/Documents/Honours/Data/Fingerprints/'
namelist = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if 'csv' and 'organ' in filename:
        pathname = directory + file
        df = pd.read_csv(pathname)
        df.drop(columns=['SMILES'], inplace=True)
        df = setbalance(df)
        datasets.append(df)
        namelist.append(filename)

sets = 0
finalresults = []
for df in datasets:
    df = df.dropna()
    dfarray = df.to_numpy()
    
    dfarray, tempset = train_test_split(dfarray, test_size=0.2, 
                                        random_state=seed, stratify = df['Toxicity_Value'])
    
    size = len(dfarray) / 5
    splitsize = math.ceil(size)
    empty = [[] for _ in range(5)]
    splits = []
    for list in empty:
        splits.append(list)
        
    loops = 0
    currsplit = 0
    #Results in a list of 5 lists that each contain 1/5 of the targetdata
    for row in dfarray:
        splits[currsplit].append(row)
        if loops == splitsize:
            loops = 0
            currsplit = currsplit + 1
        loops = loops + 1

    folds = []
    for split in splits:
        temp = pd.DataFrame(split)
        folds.append(temp)
    
    model_list = []
    validlist = []
    totalmetrics = []
    testdat = []
    #For loop that uses each fold once for valid/testing and the rest for training
    #Each 'split' in the range corresponds to the set used for test/validation with the other 4 for training
    iteration = 1
    print('========================================')
    print('For dataset', namelist[sets])
    for split in range(0,5):
        #Set creation
        trainlist = [df for i, df in enumerate(folds) if i != split]
        trainset = pd.concat(trainlist, axis=0)
        testset, validset = train_test_split(folds[split], test_size=0.5, random_state=seed)

        ytrain = trainset[0].values
        xtrain = trainset.iloc[:, 1:]

        ytest = testset[0].values
        xtest = testset.iloc[:, 1:]

        yvalid = validset[0].values
        xvalid = validset.iloc[:, 1:]

        #lists of metric values
        mcclist = []
        bestmetrics = 1
        maxval = 0
        for mtry in range(1, 50):
            #using mtry as the adjusted hyperparameter creates a series of random forests
            rf = RandomForestClassifier(n_estimators=mtry, criterion='entropy', max_depth=None, 
                                    min_samples_split=2, min_samples_leaf=1, 
                                    min_weight_fraction_leaf=0.0, max_features='sqrt', 
                                    max_leaf_nodes=None, min_impurity_decrease=0.0, 
                                    bootstrap=True, oob_score=False, n_jobs= 4, random_state=seed, 
                                    verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)

            model= rf.fit(xtrain, ytrain)
            preds = model.predict(xtest)
            #calculate metric (mcc)
            mcc = metriccalc(preds, ytest)[12]
            mcclist.append(mcc)
            if mcc > maxval:
                maxval = mcc
                bestmetrics = mtry
        #store best model for the given fold and plot the metric vs mcc value
        rf = RandomForestClassifier(n_estimators=bestmetrics, criterion='entropy', max_depth=None, 
                                    min_samples_split=2, min_samples_leaf=1, 
                                    min_weight_fraction_leaf=0.0, max_features='sqrt', 
                                    max_leaf_nodes=None, min_impurity_decrease=0.0, 
                                    bootstrap=True, oob_score=False, n_jobs= 4, random_state=seed, 
                                    verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)
        model_list.append(rf)
        testdat.append(maxval)
        totalmetrics.append(mcclist)

        #check models onto validation set, printing various metrics
        model= rf.fit(xtrain, ytrain)
        preds = model.predict(xvalid)
        results = metriccalc(preds, yvalid)
        validlist.append(results)
        print('for fold', split + 1, 'test set mcc of', maxval, 'valid set mcc of', results[12])
        
        iteration = iteration + 1
        
        #get x and y values for the final validation set

    toxvals = []
    fingerprints = []
    for row in tempset:
        toxvals.append(row[0])
        fingerprints.append(row[1:])
        
    #calculates consensus of models on each fingerprint in the final validation set
    consensuslist = []
    predictions = []
    for fp in fingerprints:
        consensus = -1
        fp = fp.reshape(1, -1)
        #appends each model's prediction to a list
        predictions = []
        predlist = []
        for model in model_list:
            preds = model.predict(fp)
            predictions.append(preds)
            predlist.append(preds)
        predictions.append(predlist)

        #finds number of 0s in the prediction list
        zercount = 0
        for num in predictions:
            if num == 0:
                zercount = zercount + 1
        #as there are 5 models, if there are less than 3 0s predicted, the consensus is 1
        if zercount < 3 :
            consensus = 1
        else:
            consensus = 0
        consensuslist.append(consensus)
    #calculate and print metrics
    results = metriccalc(consensuslist, toxvals)
    print('\nvalidation metrics of:')
    print('positives in data', results[0])
    print('negatives in data', results[1])
    print('net accuracy =', results[6])
    print('mcc =',results[12])
    finalresults.append(results[12])
    sets += 1

For dataset SR-ATAD5_Morganfp.csv
for fold 1 test set mcc of 0.5893689584893059 valid set mcc of 0.28867513459481287
for fold 2 test set mcc of 0.6077927272100749 valid set mcc of 0.2722900593672232
for fold 3 test set mcc of 0.64607127179228 valid set mcc of 0.5138888888888888
for fold 4 test set mcc of 0.4879594489999274 valid set mcc of 0.5773502691896257
for fold 5 test set mcc of 0.34877605543113943 valid set mcc of 0.49333333333333335

validation metrics of:
positives in data 51
negatives in data 53
net accuracy = 0.7307692307692307
mcc = 0.46325113915401156
For dataset NR-AhR_Morganfp.csv
for fold 1 test set mcc of 0.6892889335554799 valid set mcc of 0.52
for fold 2 test set mcc of 0.5361834956419664 valid set mcc of 0.45022516889074815
for fold 3 test set mcc of 0.6251326146494808 valid set mcc of 0.6205281366778036
for fold 4 test set mcc of 0.675363457216666 valid set mcc of 0.5500192414050356
for fold 5 test set mcc of 0.6725585057370782 valid set mcc of 0.6368430022967195




validation metrics of:
positives in data 47
negatives in data 49
net accuracy = 0.78125
mcc = 0.5818429184443191
For dataset Ames_Mutagenicity_Morganfp.csv
for fold 1 test set mcc of 0.603650250611146 valid set mcc of 0.6195795147708203
for fold 2 test set mcc of 0.6823639245219152 valid set mcc of 0.6767516838677182
for fold 3 test set mcc of 0.6954543584619967 valid set mcc of 0.6245460541922347
for fold 4 test set mcc of 0.6687298973476922 valid set mcc of 0.6481501753886127
for fold 5 test set mcc of 0.6287947720990493 valid set mcc of 0.5994738355453837

validation metrics of:
positives in data 839
negatives in data 658
net accuracy = 0.8203072812291249
mcc = 0.634433058924186
For dataset Respiratory_Toxicity_Morganfp.csv
for fold 1 test set mcc of 0.5133880625403892 valid set mcc of 0.486464257689751
for fold 2 test set mcc of 0.5595637103925328 valid set mcc of 0.5469363768841913
for fold 3 test set mcc of 0.6320981451910299 valid set mcc of 0.5153044861884355
for fold 4 test s

In [2]:
len(finalresults)

22

In [3]:
temp = pd.DataFrame(finalresults, columns=['MCC'])
temp2 = pd.DataFrame(namelist, columns=['Dataset'])
resultframe = pd.concat([temp2, temp], axis=1)
resultframe.to_csv('/Users/james/Documents/Honours/Results/Model_Outputs/RF_Fingerprint_Results.csv')